In [ ]:
!pip install -q openai chromadb --force

In [ ]:
import os
import base64
from openai import OpenAI
from google.colab import userdata
from PIL import Image
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

In [ ]:
FRAMES_DIR = "data/middleframes"
DB_PATH = "./gif_vector_store"
JSON_PATH = "data/index.json"

In [ ]:
# @title extract the middleframes.zip
import zipfile

zip_file_path = "/content/middleframes.zip"
extract_dir = FRAMES_DIR # Using the predefined GIF_ROOT_DIR

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract all contents from the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Successfully extracted '{zip_file_path}' to '{extract_dir}'")

Successfully extracted '/content/middleframes.zip' to 'data/middleframes'


In [ ]:
client = OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=userdata.get('GROQ_API_KEY'),
    # base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    # api_key=userdata.get('GOOGLE_API_KEY'),
)

In [ ]:
chroma_client = chromadb.CloudClient(
  api_key=userdata.get('CHROMADB_API_KEY'),
  tenant='e9307924-f0c2-4384-b785-43ff49b8509f',
  database='test'
)
collection = chroma_client.get_or_create_collection(name="molmo_gif_db")
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
import json

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

with open(JSON_PATH, 'r') as f:
    data = json.load(f)

In [ ]:
def get_molmo_description(image_path):
    with open(image_path, "rb") as f:
        base64_image = base64.b64encode(f.read()).decode('utf-8')

    response = client.chat.completions.create(
        model="meta-llama/llama-4-scout-17b-16e-instruct",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": "Deeply analyze the image and Describe the main expression (forexample: surprised, angry and soo on), action and subject in this GIF from giphy. Return output should be minimum 40-50 words. keyword-rich, no filler words. \n Dont use markdown formating."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]
        }]
    )
    return response.choices[0].message.content

In [ ]:
get_molmo_description("/content/data/middleframes/angry-cat-mad-this-cannot-continue-meme-ckcs1cX2tCk62n0Jzk_middle.jpg")

'The cat in the image displays a disapproving and concerned expression, with a serious and stern facial demeanor. The main action is the cat sitting upright, conveying a sense of discontent. The subject is a black and white cat with an unimpressed gaze, implying disappointment. Its eyes are narrowed, conveying skepticism and dissatisfaction.'

In [ ]:
import time

for entry in tqdm(data):
    # Construct path based on your existing middleframe file naming
    # Assuming frames are in FRAMES_DIR and named {slug}.jpg
    frame_path = os.path.join(FRAMES_DIR, f"{entry['slug']}_middle.jpg")

    # Verify the file actually exists before calling the API
    if not os.path.exists(frame_path):
        print(f"⚠️ Frame missing for: {entry['slug']} at {frame_path}")
        continue

    # Step B: Get AI Description via OpenRouter using the existing JPG
    try:
        # Check if description already exists to avoid double-spending API tokens
        current_description = entry.get('description', "")
        if current_description and current_description.strip():
            print(f"⏩ Skipping {entry['slug']}, already described.")
            description = entry['description']
            continue;
        else:
            description = get_molmo_description(frame_path)
            # Step C: Update JSON Entry
            entry['middleframe'] = frame_path
            entry['description'] = description

        # Step D: Update Vector Store (ChromaDB)
        # We do this every time to ensure the DB is in sync with the JSON
        vector = embed_model.encode(description).tolist()
        print(description)
        collection.add(
            embeddings=[vector],
            documents=[description],
            metadatas=[{
                "slug": entry['slug'],
                "expression": entry['expression'],
                "frame_path": frame_path
            }],
            ids=[entry['slug']]
        )

        time.sleep(6)
    except Exception as e:
        print(f"❌ Error on {entry['slug']}: {e}")
        break;

# 3. Save the fully updated index.json
with open(JSON_PATH, 'w') as f:
    json.dump(data, f, indent=2)

print("Done! JSON updated and Vector Store synced using existing middleframes.")


100%|██████████| 172/172 [00:00<00:00, 34338.63it/s]

⏩ Skipping mahi-memes-truly-free-human-sprit-oEHaggZvfS1SxsTNkM, already described.
⏩ Skipping cat-memes-therichnate-larry-the-p0ydOvZ6xm8PMe5qlr, already described.
⏩ Skipping wishes-midnight-new-years-greetings-BO7p37KdlUaXA5nhWO, already described.
⏩ Skipping dance-dancing-jumping-C8ih1zB9e9KJLySClx, already described.
⏩ Skipping cats-funny-cat-dancing-pAHAgWYYjWIE9DNLcC, already described.
⏩ Skipping cat-nub-nubcat-TKa7fQzChHylCQ89to, already described.
⏩ Skipping cats-memes-funny-cat-pY8jLmZw0ElqvVeRH4, already described.
⏩ Skipping f-abdullahmk47-fnaf-kitten-8hGGAmA60LsaTYdarh, already described.
⏩ Skipping cat-meme-wilfrosty-mr-fresh-wr7oA0rSjnWuiLJOY5, already described.
⏩ Skipping memes-kittens-cute-cat-angry-face-OHRF8LZis06OiPDJby, already described.
⏩ Skipping zazu-war-cat-2zUn8hAwJwG4abiS0p, already described.
⏩ Skipping memes-cat-staring-YhYgIyRd7dETRsOtK6, already described.
⏩ Skipping OJpdC50gYNJxNDEn0T, already described.
⏩ Skipping no-cat-nR4L10XlJcSeQ, already descri